In [2]:
import os
import re
from pprint import pprint
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import AffinityPropagation, DBSCAN, KMeans
from sklearn.covariance import GraphicalLassoCV, LedoitWolf, EmpiricalCovariance


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline

In [3]:
def remove_non_numbers(cnpj):
    pattern = "\D"
    return re.sub(pattern, '', cnpj)

In [4]:
df = pd.read_csv('cadastro/inf_cadastral_fi_20200221.csv', sep=';', encoding='latin-1')

In [5]:
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].apply(remove_non_numbers)

In [6]:
df.head(3)

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,DT_INI_EXERC,DT_FIM_EXERC,CLASSE,DT_INI_CLASSE,RENTAB_FUNDO,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,TRIB_LPRAZO,INVEST_QUALIF,TAXA_PERFM,INF_TAXA_PERFM,TAXA_ADM,INF_TAXA_ADM,VL_PATRIM_LIQ,DT_PATRIM_LIQ,DIRETOR,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR
0,33913620000170,051 BRUCUTU FUNDO DE INVESTIMENTO MULTIMERCADO...,2019-07-19,2019-06-04,NaN,EM FUNCIONAMENTO NORMAL,2019-08-05,2019-08-05,2019-08-05,2020-05-31,Fundo Multimercado,2019-06-04,DI de um dia,Fechado,N,S,NaN,S,20.0,NaN,0.2,NaN,25687491.42,2020-02-19,MARCELO VIEIRA FRANCISCO,62.318.407/0001-19,SANTANDER SECURITIES SERVICES BRASIL DTVM S.A,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,62.318.407/0001-19,SANTANDER SECURITIES SERVICES BRASIL DTVM S.A,62.318.407/0001-19,SANTANDER SECURITIES SERVICES BRASIL DTVM S.A
1,31964019000181,051 CAPITAL - COLORADO FUNDO DE INVESTIMENTO M...,2018-11-29,2018-11-27,NaN,EM FUNCIONAMENTO NORMAL,2018-12-10,2018-12-10,2019-07-01,2020-06-30,Fundo Multimercado,2018-11-27,DI de um dia,Fechado,N,S,S,S,0.0,NaN,0.4,NaN,18608552.02,2020-02-19,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
2,30509213000104,051 CARIBE FUNDO DE INVESTIMENTO MULTIMERCADO ...,2019-06-11,2018-04-13,NaN,EM FUNCIONAMENTO NORMAL,2019-06-25,2019-06-25,2020-02-01,2021-01-31,Fundo Multimercado,2018-04-13,DI de um dia,Fechado,N,S,S,S,8.0,NaN,0.2,NaN,41495388.58,2020-02-19,MARCELO VIEIRA FRANCISCO,62.318.407/0001-19,SANTANDER SECURITIES SERVICES BRASIL DTVM S.A,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES,62.318.407/0001-19,SANTANDER SECURITIES SERVICES BRASIL DTVM S.A,62.318.407/0001-19,SANTANDER SECURITIES SERVICES BRASIL DTVM S.A


In [7]:
df['MAIS_CEM'] = np.nan

for cnpj in df.CNPJ_FUNDO:
    try:
        df_fund = pd.read_csv('funds/{}.csv'.format(cnpj), index_col='DT_COMPTC', parse_dates=True)
        num_cotistas = df_fund.iloc[-1].NR_COTST
        if num_cotistas >= 100:
            df.loc[df.CNPJ_FUNDO == str(cnpj), 'MAIS_CEM'] = 'S'
        else:
            df.loc[df.CNPJ_FUNDO == str(cnpj), 'MAIS_CEM'] = 'N'
    except FileNotFoundError:
        df.loc[df.CNPJ_FUNDO == str(cnpj), 'MAIS_CEM'] = np.nan

In [8]:
df = df.drop_duplicates(subset=['CNPJ_FUNDO'], keep='last')

In [9]:
df_clusterable = df[
    (df.SIT == 'EM FUNCIONAMENTO NORMAL') & (df.FUNDO_EXCLUSIVO == 'N') & (df.DT_INI_SIT <= '2019-01-01') & (df.MAIS_CEM == 'S')
]

In [10]:
df_clusterable.shape

(2127, 37)

In [11]:
df_clusterable.CLASSE.value_counts()

Fundo Multimercado     854
Fundo de Renda Fixa    758
Fundo de Ações         489
Fundo Cambial           26
Name: CLASSE, dtype: int64

In [12]:
dict_cnpjs = {}

for classe in df_clusterable.CLASSE.unique():
    dict_cnpjs[classe] = df_clusterable[df_clusterable.CLASSE == classe]['CNPJ_FUNDO'].tolist()

In [13]:
def create_dataframe(cnpjs, start='2019-01-01', end='2019-12-31', freq='B', bar='VL_QUOTA', folder='funds'):
    index = pd.date_range(start=start, end=end, freq=freq)
    df = pd.DataFrame(index=index)
    
    for cnpj in cnpjs:
        df_tmp = pd.read_csv(
            '{}/{}.csv'.format(folder, cnpj),
            index_col='DT_COMPTC',
            parse_dates=True,
            na_values=['null', 'N/A', 'NULL']
        )
        df_bar = df_tmp[[bar]]
        df_bar.columns = [cnpj]
        df = df.join(df_bar, how='left')
        
    return df

In [14]:
df_quotas = create_dataframe(df_clusterable.CNPJ_FUNDO)

In [16]:
def fill_missing(df):
    df = df.replace([np.inf, -np.inf], np.nan)
    return df.fillna(method='ffill').fillna(method='bfill')

def normalize_data(df):
    df = fill_missing(df)
    return fill_missing(df/df.iloc[0])

def get_log_returns(df):
    df = fill_missing(df)
    df = np.log(df) - np.log(df.shift(1))
    return fill_missing(df)

In [17]:
df_quotas.shape

(261, 2127)

In [18]:
df_rets = get_log_returns(normalize_data(df_quotas))

/home/igor/.local/share/virtualenvs/scripts-I1e1_A0A/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


In [19]:
df_rets.head()

,31095996000190,22232929000189,23657955000111,28581653000120,09599346000122,09550197000107,04018864000191,01353626000144,29298535000172,18860180000147,21983042000160,18860059000115,31594616000161,32073525000143,28947266000165,18422272000145,29177012000178,23565803000199,31238336000110,18214066000140,26831986000135,24413725000170,28750204000169,24413703000100,24247636000109,24976673000140,28750061000195,28021860000120,26419969000195,24986039000198,26622780000103,30077777000115,26324098000126,24119419000125,24078037000109,24029438000160,25382472000187,29298529000115,23979285000150,29011235000160,29286406000164,23951048000180,29137829000112,27739651000154,09720734000110,29044189000104,11209172000196,08909429000108,12987743000186,26648868000196,26673556000132,13001211000190,47177431000191,00575922000127,08254141000142,19154041000160,02733791000194,28504521000103,56824857000180,28839048000107,23352177000152,12120470000178,12120464000110,12154392000122,12120475000109,00089915000115,04874817000140,27770685000101,23034819000175,30910036000173,09620860000100,18602405000165,15862830000103,13608337000128,17397804000179,27826024000150,28798135000163,19523641000159,24215286000190,21254851000130,20441301000168,09625909000100,30921203000181,30910199000156,17414721000140,32102131000176,06041290000106,04515848000104,32225663000109,03168062000103,07158060000186,10237480000162,03369187000193,31159166000189,30910553000142,17249227000178,23186344000132,28075454000140,13083227000190,11145320000156,19726267000199,21762107000147,18585301000190,17012208000123,31455915000115,07279657000189,22100009000107,26387057000189,25213405000139,09141893000160,31288760000170,10320188000109,23556185000110,23720500000101,27077299000139,04455632000109,26323079000185,11392165000172,22681798000117,13974750000106,14812722000155,19782311000188,16599968000116,24669263000156,30045296000128,30045263000188,26718237000104,08799265000103,28692436000108,08627607000108,09528698000197,07812369000148,23732231000195,17489031000150,28747733000103,29733842000134,17087932000116,28235629000130,21228297000117,30521517000197,28140892000145,28747722000123,30556644000121,09635172000106,03897077000102,26104747000183,30378445000170,08018849000102,05357507000110,00838269000141,36347706000171,00838267000152,20230719000126,19170160000107,01587403000141,00838266000108,00857361000159,21743480000150,88198056000143,01353260000103,10199934000158,08960978000107,16844934000140,21007180000103,16844890000158,18466245000174,16844885000145,02131725000144,02131724000108,01822655000108,08960570000135,04828795000181,08960557000186,97261093000140,08960971000195,10199942000102,11311863000104,23863573000144,11311874000186,02430487000178,00743026000120,06251554000148,18270783000199,09134614000130,17593934000187,15519316000161,10869628000181,09648050000154,08973942000168,05100191000187,14213331000114,04312909000136,18690918000175,02020528000158,30530779000118,29967641000100,05100213000109,28588902000100,22632237000128,10418335000188,00822059000165,73899759000121,09005823000184,30847180000102,09004364000114,06349816000101,11328904000167,29979814000100,07882792000114,04222716000194,03920413000182,30518554000146,09005805000100,27565832000101,08973948000135,08973951000159,05100221000155,01578474000188,04881682000140,29258294000138,04965504000105,00073041000108,00842960000107,03213328000184,04128893000106,04061044000182,04305193000140,19689941000102,13482668000164,30629260000191,30506340000150,28015594000123,17940400000180,26751296000176,30506347000171,26739055000101,20162345000159,29291533000151,13449021000130,26732468000164,20162353000103,30518545000155,30506330000114,04128522000124,02998773000134,29291519000158,13079634000123,11046635000146,10869591000191,11802604000178,...,30329393000142,55075238000178,12884390000199,16617768000149,31341360000180,24572061000191,12831360000114,18507745000107,29127763000180,28797765000113,28912489000197,12809201000113,09285146000103,09412822000154,16478741000112,30338659000113,30282

In [20]:
def rescaling(df):
    mean = df.mean()
    std = df.std()
    return (df - mean)/std

In [21]:
df_rescaled = rescaling(df_rets)

In [67]:
class BranchKmeans:
    def __init__(self, n_min, n_max, n_iter):
        self.min = n_min
        self.max = n_max
        self.iter = n_iter
        self.lables_ = None
        self.num = int((self.min + self.max)/2)
        self.news = []
        self.bigs = None
        self.original_labels = None
        self.new_labels = None
    
    def fit(self, df):
        n_clusters = self.get_num_clusters(df)
        clustering = KMeans(n_clusters=n_clusters, max_iter=1000).fit(df.values.T)
        self.original_labels = clustering.labels_.tolist()
        self.mew_labels = self.original_labels
        
        clusters = dict()
        
        for label in set(clustering.labels_):
            clusters[label] = []
            
        for cnpj, label in zip(list(df), clustering.labels_):
            clusters[label].append(cnpj)
                    
        big_clusters = []
        for label in set(clustering.labels_):
            if len(clusters[label]) > self.max:
                big_clusters.append(label)
             
        self.bigs = big_clusters    
            
        for big in big_clusters:
            cnpjs = clusters[big]
            new_df = df[cnpjs]
            
            new_n_clusters = self.get_num_clusters(new_df)
            new_clustering = KMeans(n_clusters=new_n_clusters, max_iter=1000).fit(new_df.values.T)
            
            self.news.append(new_clustering.labels_.tolist())
        

        
        return self
    
    def get_num_clusters(self, df):
        m = len(list(df))
        return int(m/self.num)
    

In [68]:
bkmeans = BranchKmeans(3,10, 10)

In [69]:
bkmeans.fit(df_rescaled)

In [72]:
len(bkmeans.bigs)

38

In [71]:
len(bkmeans.news)

38

In [ ]:
for l

In [44]:
# clustering = AffinityPropagation(max_iter=300).fit(df_rescaled.values.T)
clustering = KMeans(n_clusters=532, max_iter=2000).fit(df_rescaled.values.T)
# clustering = DBSCAN(min_samples=5).fit(df_rescaled.values.T)

In [45]:
for cnpj, label in zip(list(df_rets), clustering.labels_):
#     if label == 487:
    print(cnpj, label)

31095996000190 346
22232929000189 175
23657955000111 520
28581653000120 509
09599346000122 446
09550197000107 463
04018864000191 276
01353626000144 272
29298535000172 201
18860180000147 529
21983042000160 146
18860059000115 40
31594616000161 387
32073525000143 387
28947266000165 40
18422272000145 40
29177012000178 40
23565803000199 40
31238336000110 40
18214066000140 469
26831986000135 228
24413725000170 8
28750204000169 8
24413703000100 264
24247636000109 9
24976673000140 19
28750061000195 54
28021860000120 94
26419969000195 36
24986039000198 60
26622780000103 5
30077777000115 356
26324098000126 8
24119419000125 8
24078037000109 8
24029438000160 8
25382472000187 8
29298529000115 8
23979285000150 8
29011235000160 8
29286406000164 8
23951048000180 8
29137829000112 8
27739651000154 8
09720734000110 421
29044189000104 421
11209172000196 361
08909429000108 361
12987743000186 372
26648868000196 372
26673556000132 372
13001211000190 240
47177431000191 361
00575922000127 361
08254141000142 57

27566556000104 41
27566566000131 33
30658954000157 48
30658606000180 48
26570686000140 110
28651543000198 102
20077065000142 410
09296022000115 480
73970642000197 108
07187591000105 515
08610270000126 389
03515596000150 14
08893172000143 119
28419514000103 57
18048639000102 440
27771966000189 119
05903038000198 119
68971183000126 437
05736300000157 25
18799585000117 17
14632925000160 496
30598058000140 488
00598452000117 42
04570232000136 42
01656101000188 205
07899238000140 57
30773649000106 154
31340883000102 154
28386218000145 154
28407875000121 154
09577098000119 169
13554383000191 24
18908578000106 176
24140256000162 344
20658576000158 480
06893041000130 160
19186860000190 2
06118222000190 2
13033721000140 505
22919232000180 327
19488768000184 517
14115118000170 24
18683208000118 440
11435287000107 347
15585932000110 269
14796095000106 488
23600995000127 296
14816882000172 430
17404144000106 361
04118043000127 74
27746867000147 219
13608335000139 361
09586692000176 208
29762338000

In [46]:
cnt = Counter(clustering.labels_)

In [48]:
cnt[320]

2

In [49]:
singles = []

for cnpj, label in zip(list(df_rets), clustering.labels_):
    if cnt[label] == 1:
        singles.append(cnpj)

In [50]:
len(singles)

287

In [51]:
clustering_singles = KMeans(n_clusters=int(287/4), max_iter=2000).fit(df_rescaled[singles].values.T)


In [52]:
for cnpj, label in zip(list(df_rets[singles]), clustering_singles.labels_):
#     if label == 487:
    print(cnpj, label)

31095996000190 6
22232929000189 58
23657955000111 1
28581653000120 51
09550197000107 7
29298535000172 43
18860180000147 34
21983042000160 34
18214066000140 51
13001211000190 14
02733791000194 29
27770685000101 29
09620860000100 24
19523641000159 67
21254851000130 47
09625909000100 63
17414721000140 51
32225663000109 12
07158060000186 39
17012208000123 22
26387057000189 14
10320188000109 48
22681798000117 28
19782311000188 29
30045296000128 22
00857361000159 14
01353260000103 15
16844934000140 2
02430487000178 28
15519316000161 7
28588902000100 17
29979814000100 7
08973951000159 67
01578474000188 7
04305193000140 46
13079634000123 13
11802604000178 6
06015361000198 13
15494363000106 13
13083185000197 6
19664603000116 43
10925558000131 33
00360293000118 46
04260468000176 14
29215547000196 13
18428661000188 45
03996663000104 2
21065857000160 21
06124241000129 14
13207280000155 56
18386802000147 67
18820151000151 65
20283162000191 2
20658901000182 62
08823534000120 64
23379114000190 37
233

In [1]:
df_rets

NameError: name 'df_rets' is not defined

In [25]:
v = dict()

In [26]:
v[1] = 2

In [27]:
v

{1: 2}